In [51]:
import requests
from fastbook import *
def search_images_bing_new(key,term,customConfigId,min_sz=128):    
    url = 'https://api.bing.microsoft.com/v7.0/custom/images/search?' + 'q=' + term + '&' + 'customconfig=' + customConfigId + '&' + 'count=150'
    r = requests.get(url, headers={'Ocp-Apim-Subscription-Key':key})
    search_results = r.json()    
    print(term)
    return L([img["thumbnailUrl"] for img in search_results["value"][:150]])

In [52]:
search_images_bing_new("1be3ada0f2a649d087e9fb39798485a0", "pizza", "efb1b149-c219-4091-be6d-14a279c405c4")

pizza


(#150) ['https://tse3.mm.bing.net/th?id=OIP.Tgc4qUDMrJylw_EWYbMUtQHaFS&pid=Api','https://tse4.mm.bing.net/th?id=OIP.XF7m9Bf6YeFD7qCiUk9_lgHaEK&pid=Api','https://tse4.mm.bing.net/th?id=OIP.JCMHp14SQtQwD-ykfDENgwFNC7&pid=Api','https://tse1.mm.bing.net/th?id=OIP.1R4hze9Sou3MQzoXTw8aBgHaFj&pid=Api','https://tse4.mm.bing.net/th?id=OIP.Zyu9zfxQbFiZfqGiicsqUgHaEK&pid=Api','https://tse3.mm.bing.net/th?id=OIP.0xEAKC9dw8mMlKHsMDZKigAAAA&pid=Api','https://tse1.mm.bing.net/th?id=OIP.717sO-yuwZRsER5eJl50JgHaEK&pid=Api','https://tse2.mm.bing.net/th?id=OIP.tiKoPew1IkBDsM_FCqyLKQHaEK&pid=Api','https://tse2.mm.bing.net/th?id=OIP.lqb2vBwRvjxu10gocDiHIQHaFj&pid=Api','https://tse1.mm.bing.net/th?id=OIP.pPdkZHeCLXulRdSsOArLhAHaFj&pid=Api'...]

In [69]:
painter_types = "C. G. Kratzenstein Stub", "Nicolai Abildgaard"
path = Path('painter')
if not path.exists():
    print("plz")
    path.mkdir()
    for o in painter_types:
        print(f'{o}')
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing_new("1be3ada0f2a649d087e9fb39798485a0", o, "efb1b149-c219-4091-be6d-14a279c405c4")
        download_images(dest, urls=results)

plz
C. G. Kratzenstein Stub
C. G. Kratzenstein Stub


Nicolai Abildgaard
Nicolai Abildgaard


In [66]:
fns = get_image_files(path)
fns

(#300) [Path('painter/hotdog/00000000.jpg'),Path('painter/hotdog/00000007.jpg'),Path('painter/hotdog/00000005.jpg'),Path('painter/hotdog/00000001.jpg'),Path('painter/hotdog/00000002.jpg'),Path('painter/hotdog/00000006.jpg'),Path('painter/hotdog/00000004.jpg'),Path('painter/hotdog/00000003.jpg'),Path('painter/hotdog/00000012.jpg'),Path('painter/hotdog/00000011.jpg')...]

In [57]:
failed = verify_images(fns)
failed

(#0) []

In [58]:
failed.map(path.unlink)

(#0) []

In [61]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders): self.loaders = loaders
    def __getitem__(self,i): return self.loaders[i]
    train,valid = add_props(lambda i,self: self[i])
    
food = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))



In [62]:
food = food.new(
    item_tfms=RandomResizedCrop(224,min_scale=0.5),
    batch_tfms=aug_transforms())
dls = food.dataloaders(path)

In [64]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

epoch,train_loss,valid_loss,error_rate,time
0,1.005386,0.636154,0.266667,00:02


epoch,train_loss,valid_loss,error_rate,time
0,0.395623,0.181591,0.083333,00:02
1,0.337682,0.049772,0.000000,00:02
2,0.250209,0.024089,0.000000,00:02
3,0.204521,0.015663,0.000000,00:02
4,0.162210,0.009398,0.000000,00:02
5,0.133787,0.006265,0.000000,00:02
6,0.112834,0.005773,0.000000,00:02
7,0.096246,0.007061,0.000000,00:02
8,0.085900,0.010081,0.000000,00:02
9,0.079036,0.011370,0.000000,00:02
